<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/Experimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bibliotecas 

In [ ]:
#Importing needed libraries 
!pip install -U scikit-learn

from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd #Biblioteca para carregamento de arquivo csv
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter


#NLTK tools for text processing 

import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


#Modeling packages 

from sklearn.model_selection import train_test_split
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'



# Experimento com a HateBR 1 (com pré-processamento)



In [22]:
from pandas.core.groupby import base
base_hatebr = pd.read_csv('/content/drive/MyDrive/Dataset/HateBR.csv')
base_hatebr[:5]

,instagram_comments,offensive_language,offensiveness_levels,hate_speech
0,este lixo ...,1,1,-1
1,Mais um lixo,1,1,-1
2,Essa nao tem vergonha na cara!!,1,2,-1
3,Essa mulher é doente.pilantra!,1,3,-1
4,Comunista safada...,1,2,"5,8"


In [21]:
base_hatebr.offensive_language.value_counts()

1    3500
0    3500
Name: offensive_language, dtype: int64

Dividindo em base de teste e base de treinamento



In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    base_hatebr.instagram_comments,
    base_hatebr.offensive_language, 
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = base_hatebr.offensive_language
)

In [26]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (5600,)
Shape of X_test (1400,)


In [27]:
y_train.value_counts()

1    2800
0    2800
Name: offensive_language, dtype: int64

In [28]:
y_test.value_counts()

1    700
0    700
Name: offensive_language, dtype: int64

Teste com Naive Bayes




In [34]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.85      0.81      0.83       700
           1       0.82      0.86      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



Remoção de Stop Words


In [3]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in base_hatebr ['instagram_comments']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))
  

Lematizando com NLTK

In [4]:
  for i in range (len(comments)):
    words = nltk.word_tokenize(comments[i])
    newwords = [lemmatizer.lemmatize(word) for word in words]
    comments[i] = ' '.join(newwords)
print (comments)

['lixo', 'lixo', 'nao vergonha cara', 'mulher doente pilantra', 'comunista safada', 'vagabunda comunista mentirosa povo chileno nao merece desgra desta', 'comunista safada malparida mamagueva desgra ada co madre', 'besta quadrada', 'comunista lixo', 'hipocritas', 'hipocrita', 'pena galinha vaca louca', 'esquerdista', 'mande bachelet plantar batata asfalto puta pariu at tamborete bordel acha direito falar brasil', 'mulherzinha safada', 'cara pau', 'oportunista corrupta agora todos comunistas querem fazer v timas', 'porque bandida', 'mentem roubam n vai mamar', 'tudo igual s pensam pr prio rabo', 'plano esquerda sempre cr ticar n nunca plano governo nico plano roubar', 'cretina', 'guarde pena nsia v mito', 'n t m car ter contra presidente bolsonaro', 'problema tetinha secou', 'algu m s consci ncia ainda vai acreditar nesta farsante n acredito fio cabelo pintado', 'bandida', 'cara homem n macron deve achar gata', 'cara pau', 'nojo dessa pilantra', 'olha ai hipocrisia dessa mulher', 'bando

In [5]:
v = CountVectorizer(ngram_range=(1,1))
v.fit(comments)
comentarios_vetor = v.transform(comments).toarray()
comentarios_vetor

CountVectorizer()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

O TF-IDF mede a importância de uma determinada palavra em relação a um documento e a todo o corpus.

In [6]:
tfidf = TfidfVectorizer()
transformed_output = tfidf.fit_transform(comments)
print(tfidf.vocabulary_)

{'lixo': 5424, 'nao': 6126, 'vergonha': 9186, 'cara': 1475, 'mulher': 6089, 'doente': 2973, 'pilantra': 6947, 'comunista': 1930, 'safada': 8091, 'vagabunda': 9056, 'mentirosa': 5808, 'povo': 7149, 'chileno': 1672, 'merece': 5826, 'desgra': 2651, 'desta': 2717, 'malparida': 5580, 'mamagueva': 5586, 'ada': 150, 'co': 1765, 'madre': 5547, 'besta': 1044, 'quadrada': 7517, 'hipocritas': 4518, 'hipocrita': 4517, 'pena': 6766, 'galinha': 4221, 'vaca': 9046, 'louca': 5452, 'esquerdista': 3485, 'mande': 5607, 'bachelet': 911, 'plantar': 7001, 'batata': 998, 'asfalto': 723, 'puta': 7490, 'pariu': 6638, 'at': 779, 'tamborete': 8599, 'bordel': 1181, 'acha': 83, 'direito': 2858, 'falar': 3745, 'brasil': 1228, 'mulherzinha': 6091, 'pau': 6701, 'oportunista': 6461, 'corrupta': 2189, 'agora': 252, 'todos': 8765, 'comunistas': 1931, 'querem': 7568, 'fazer': 3844, 'timas': 8730, 'porque': 7096, 'bandida': 952, 'mentem': 5802, 'roubam': 8039, 'vai': 9060, 'mamar': 5588, 'tudo': 8953, 'igual': 4639, 'pens

In [7]:
#impressão na ordem
tfidf.get_feature_names_out()

array(['aaah', 'ab', 'abacaxi', ..., 'zozibini', 'zum', 'zzzz'],
      dtype=object)

In [ ]:
all_feature_names = tfidf.get_feature_names_out()
for word in all_feature_names:
  index = tfidf.vocabulary_.get(word)
  print(f'{word} {tfidf.idf_[index]}') #A pontuação para termos que aparecem em muito soducmento é menor e a pontuação para termos raros é maior.

In [9]:
comments[:3]

['lixo', 'lixo', 'nao vergonha cara']

In [10]:
transformed_output.toarray()[:3]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Experimento com a HateBR 2 (sem pré-processamento)


In [37]:
from pandas.core.groupby import base
base_hatebr = pd.read_csv('/content/drive/MyDrive/Dataset/HateBR.csv')
base_hatebr[:5]

,instagram_comments,offensive_language,offensiveness_levels,hate_speech
0,este lixo ...,1,1,-1
1,Mais um lixo,1,1,-1
2,Essa nao tem vergonha na cara!!,1,2,-1
3,Essa mulher é doente.pilantra!,1,3,-1
4,Comunista safada...,1,2,"5,8"


In [38]:
base_hatebr.offensive_language.value_counts()

1    3500
0    3500
Name: offensive_language, dtype: int64

Dividindo em base de teste e base de treinamento


In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    base_hatebr.instagram_comments,
    base_hatebr.offensive_language, 
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = base_hatebr.offensive_language
)

In [40]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (5600,)
Shape of X_test (1400,)


In [41]:
y_train.value_counts()

1    2800
0    2800
Name: offensive_language, dtype: int64

In [42]:
y_test.value_counts()

1    700
0    700
Name: offensive_language, dtype: int64

Teste com Naive Bayes




In [43]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.85      0.81      0.83       700
           1       0.82      0.86      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



Verificando  predição

In [51]:
X_test[:5]

309     Ela teria que ter pena qndo tirou dos hospitai...
2117                                  Um grande perdedor!
4656                           Os dois homens que eu amo.
4501    Meu Deus que absurdo, presidente dele fala o q...
1009                          #Globolixo 🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣
Name: instagram_comments, dtype: object

In [50]:
y_test[:5]

309     1
2117    1
4656    0
4501    0
1009    1
Name: offensive_language, dtype: int64

predição do algoritmo 

In [52]:
y_pred[:5]

array([1, 0, 0, 0, 1])